# Decentralized Path Planning - Mini Problem Set

### Due May xth, 11:59 p.m.

In this problem set, you'll implement **DMA-RRT** and **DMA-RRT Collaborative** and use it to solve decentralized multiagent path planning problems. Specifically, you will apply it to the VIPER path planning scenario.

#### Table of Contents

TODO

Make sure you load the following dependencies by highlighting the cell below and pressing `Shift + Enter`.

In [ ]:
from solutions.plan import Plan, dma_individual, dma_coop_individual
from solutions.agent import Agent
from utils.environment import Environment

## Motivation

In brief, NASA's current architecture for lunar exploration is a gradual ramp up of capabilities. The first foray, the Volatiles Investigating Polar Exploration Rover, will launch in the 2022 timeframe with the express mission of mapping water-ice at potential Artemis landing sites. From this perspective, we can extrapolate to a concept of operations for planetary exploration wherein teams of rovers are responsible for advanced scouting. If humans are dependent on their results, time will be of the essence.

Furthermore, novel environments will introduce communication challenges. There may not be resources to run central authorities. Ground communications may be limited, spotty, or massively delayed.

Given all the constraints described above, we need to consider algorithms that allow teams of agents to distribute decision making online. For this problem set, you will address the problem of distributed path planning.

See the tutorial for more information.

## Modeling

We've provided some tools to make implementation easier. In particular, we have provided you with a working implementation of the RRT* variant of Rapidly Exploring Random Trees (RRT), which is the underlying base planner for each agent in the scenario. You will be responsible for implementing the interactive and cooperative components between agents.

In addition, there are helper functions and vizualization tools provided to help with implementing the core Decentralized, Multi-Agent RRT (DMA-RRT) algorithms. If you're curious how the code, works, feel free to check out the source files provided.

### Modeling Example

Consider the following problem:

> A VIPER-type rover is in a potential science site. We will assume car-like dynamics for the rover. It needs to move from a start location at (0, 0) to the goal site at (10, 6).

Let's model this problem with the API we provide:

In [ ]:
# create an environment of defined boundaries and obstacles
env = Environment("../utils/simple.yaml")

start_pos = [10, 4]
goal_site = [-1.7, 0]

# create a single agent
agent = Agent("normal", start_pos, goal_site, env, 0.5, 200)

The `Agent` constructor has the following API:

    mode: a string: "normal" or "cooperative" indicates whether to use DMA-RRT or Cooperative DMA-RRT (respectively).
    start_pos: a list of 2 floats representing the starting x-y position.
    goal_pos: a list of 2 floats representing the goal x-y position.
    environment: an Environment object representing the map in which the agent must plan.
    goal_dist: a float representing the length of a single branch in the RRT tree for the agent's planner.
    rrt_iters: the number of iterations to run for RRT at each spin_once

We have successfully modeled the problem; we've instantiated an `Agent` with assumed car-like dynamics, as well as an environment (2D map) in which the agent is operating. We've given the agent a start and end point for planning as well. Now all that's left is to encapsulate the problem in a `Plan` object, which works for both the single and multi-agent scenarios.

In [ ]:
# create a plan for 1+ agents
example_plan = Plan([agent], env, dma_individual, dma_coop_individual, 100, viz_trees=True, headless=False)

The `Plan` constructor has the following API:

    agents: a list containing Agent objects
    env: an Environment object
    dma_indiv: a method representing the individual component of the DMA-RRT algorithm.
    dma_coop: a method representing the cooperative extension of the DMA-RRT algorithm.
    spin_rate: the rate (in Hz) at which the planner should run
    viz_trees: if True, visualizer will show RRT trees for all agents in orange.
    headless: if True, nothing will be visualized during spin.

Now we have a `Plan`, which encapsulates agent(s), strategies for cooperation, and RRT parameters. Let's vizualize the results for a single agent. At each time step the agent generates new partial paths to the goal (in orange), compares them to their existing path (grey), and executes the path with the lowest cost.

In [ ]:
plan = Plan([agent], env, dma_individual, dma_coop_individual, 100, viz_trees=True, headless=False)

# execute the plan through at most 50 planning cycles
plan.spin(50)

Feel free to run the plan with different parameters or run it as many times as you'd like - you should see a different path created each time. Now let's analyze the behavior of this implementation of RRT*.

#### Conceptual Question 1 - RRT

While RRT is usually allowed to continue generating paths until a path to the goal is found, our implementation of RRT* has a time limit for generating paths before it must pick and execute the best one according to a Euclidean distance heuristic between the end node of each path and the goal. After it completes execution, it starts building a new tree from scratch. This means there is no guarantee that the agent is generating paths that reach the goal at any given timestep.

We'd like you to brainstorm the pros and cons of a fixed time limit for path planning in RRT. How does a fixed time limit impact the overall optimality of the complete path? In what situations would a fixed time limit be preferrable to an unlimited time limit? Why?

YOUR ANSWER HERE

---

It depends on the variant of RRT. For vanilla RRT, a fixed time limit does not impact the optimality of the system. For RRT\*, a fixed time limit will reduce the optimality of the system because it effectively prevents RRT\* from iterating on the goal path enough times to approach optimality.

Given RRT is an anytime algorithm, a fixed time limit simply reduces the amount of planning performed before execution. A fixed time limit is helpful when it is cheap, easy, and preferable to replan often.

#### Conceptual Question 2 - Coordination

Why kind of problems are created when all agents are allowed to replan independently and in parallel? Does a round-robin approach address all the issues? Why or why not?

YOUR ANSWER HERE

---

If agents replan independently and in parallel, agents will not be using guaranteed information about the paths of their peers. As such, paths may collide because agents are planning against outdated peer paths. Additionally, not all agents always need to replan. It is possible for RRT\* to produce nearly optimal plans, which means that constant replanning is not a necessity for all agents.

A round-robin approach does address the issue of stale information. If agents take turns replanning, they can be sure that they are planning against up-to-date information about their peers positions. A round-robin approach does not address the issue of unnecessary replanning, as all agents will spend equal amounts of time replanning regardless of the optimality of their current plans.

#### Conceptual Question 3 - Negative Bids

If you haven't already, now is a good time to review the DMA-RRT algorithm in the tutorial. In it, a bid is defined as the difference between the cost of the current path and the cost of the best path in the tree. With that in mind, is it possible for an agent to advertise a negative bid? Why or why not? What would a negative bid signify regarding comparative path costs?

YOUR ANSWER HERE

---

No, it is not possible for an agent to produce a negative bid. A negative bid would signify that the best plan is worse than the plan they are currently executing. Bids are only generated from plans that create cost benefits.

## Implement DMA-RRT

We have provided you with CL-RRT for each agent, but as we dicussed in lecture, having each agent run CL-RRT on its own in isolation can lead to collisions between agents. We must therefore implement DMA-RRT such that the agents can plan as a team and avoid collisions.

The `Agent` class also has an `Antenna` attribute that allows it to communicate with other agents. This is fully implemented for you; you can use the following APIs to transmit and receive messages to and from other agents:

TODO: add Antenna class API table here!

The `Agent` class also has an internal CL-RRT planner that you will need to interface with. Here is the API list for that class:

TODO: talk more about how you use this API!
TODO: add RRT class API table here!

DMA-RRT has two components. The individual component handles internal RRT tree grown, choosing the best path, and sending messages to other agents. The interaction component handles inter-agent communications and updating internal constraints. Refer to the tutorial for more information on these algorithms.

Implement DMA-RRRT individual and interaction as two separate functions below. Refer to the docstring for more information on input args and return types.

<div class="alert alert-info">
Complete the functions below.
</div>

In [ ]:
def dma_rrt_individual(self, agent):
    """ TODO: add docstring!
    """
    # YOUR CODE HERE
    raise NotImplementedError
    
    
def dma_rrt_interaction(self, agent):
    """ TODO: add docstring!
    """
    # YOUR CODE HERE
    raise NotImplementedError

Execute the cell below to visualize your planner in real time!

In [ ]:
plan.dma_rrt_indiv = dma_rrt_individual
plan.dma_rrt_inter = dma_rrt_interaction

plan.spin(viz=True)

You can check your implementation here.

In [ ]:
check_plan_viper(plan)
test_ok()

#### Conceptual Question 4 - Cooperative DMA-RRT

Cooperative DMA-RRT layers a protocol for asking other agents to alter their plans on top of DMA-RRT's bidding process. Using the notion of broadcasting "emergency stops" alongside waypoints, Cooperative DMA-RRT claims to improve the global optimality of the system. See the tutorial for more information.

Let's think about risk at a high level. How does the global optimality of a system of distributed agents impact the risk associated with path planning? Would you expect that agents would take on more risk in a more optimized system? Less risk? Why or why not?

YOUR ANSWER HERE

---

A more optimized plan is one where agents are taking more direct routes to goals. Take the idea of an street intersection as an example. In a fully optimized system, agents would slightly alter their speed to adjust their timing on approach to the intersection to narrowly avoid cross traffic. Humans are unable to do this, so instead we use a less optimized system of taking turns. This increases the distance between vehicles. In a similar way, agents in a more optimized system would be willing to create paths that approach closer to one another, hence increasing risk.

## Performance Analysis

RRT* and DMA-RRT have many parameters for performance tuning. Let's take a look at one of the more impactful - $\Delta t$, or the amount of time permitted during RRT to grow trees. $\Delta t$ corresponds to `rrt_iters`, or the number of iterations of RRT tree-growth to run during planning, in the `Agent` constructor API. In the next cells, use your implementation of DMA-RRT to experiment with the impact of modifying `rrt_iters`.

In [ ]:
rrt_iters = 200

agents = [Agent("normal", [0, 0], [10, 6], env, 0.75, rrt_iters),
          Agent("normal", [0, 5], [4, 6], env, 0.75, rrt_iters),
          Agent("normal", [4, -2], [-1.7, 0], env, 0.75, rrt_iters)]

plan = Plan(agents, env, sol_individual, sol_coop_individual, 100, viz_trees=False, headless=False)
plan.spin(75)

In [ ]:
rrt_iters = 300

agents = [Agent("normal", [0, 0], [10, 6], env, 0.75, rrt_iters),
          Agent("normal", [0, 5], [4, 6], env, 0.75, rrt_iters),
          Agent("normal", [4, -2], [-1.7, 0], env, 0.75, rrt_iters)]

plan = Plan(agents, env, sol_individual, sol_coop_individual, 100, viz_trees=False, headless=False)
plan.spin(75)

In [ ]:
rrt_iters = 400

agents = [Agent("normal", [0, 0], [10, 6], env, 0.75, rrt_iters),
          Agent("normal", [0, 5], [4, 6], env, 0.75, rrt_iters),
          Agent("normal", [4, -2], [-1.7, 0], env, 0.75, rrt_iters)]

plan = Plan(agents, env, sol_individual, sol_coop_individual, 100, viz_trees=False, headless=False)
plan.spin(75)

#### Conceptual Question 5 - The Impact of $\Delta t$

How does $\Delta t$ impact the performance of DMA-RRT? If you were implementing a real-world version of DMA-RRT, what metrics would you use to tune the planning time of your agents?

YOUR ANSWER HERE

---

As $\Delta t$ grows, DMA-RRT slows down. Agents are allowed more time to plan. With RRT\* that means the paths are potentially more optimized, but it comes at the cost of taking longer for each turn to complete.

In the real world, I would compare the time it takes to replan to the total execution time. I would increase the planning time until such a point that it was non-negligible in comparison to the execution time. I would expect that planning should be done while executing, and as such so long as there is little chance of planning taking longer than executing, I would be comfortable increasing the planning time.

#### Conceptual Question 6 - Limitations of DMA-RRT

How would DMA-RRT scale with the number of agents? We only simulated environments with a few agents in this problem set, but could it feasibly be run with 100s or 1000s of independent agents? Why or why not?

YOUR ANSWER HERE

---

The two factors that will determine the scalability of DMA-RRT is the time it takes to model peer paths and the time to bid.

Any lag or communication issues in the network of agents will dramatically impact the time to bid. Our implementation modeled a fully-connected, lag-free network of agents. So long as all agents can communicate, bidding should be a quick process regardless of the number of agents. However, any real world system would need to address communication delays and dropouts. If not, a single agent with communication issues would inhibit the bidding process, slowing down planning across all agents.

Assuming it remains computationally cheap to model the paths of other agents, then it seems likely that DMA-RRT could scale to high numbers of agents. 

## **You're Done!**
Don't forget to validate your notebook and submit.